# 2024-1 DSL 정규세션 과제

## CNN Assignment

- 작성자 : 윤형진
- 참고자료 : PyTorch Document
- 과제 제출 기한 : 02.18

---
- 이름 : 송예원    
- 기수 : 11기    



In [ ]:
!pip install wandb

In [2]:
# colab 환경에서 학습을 진행하실 분들은 구글드라이브를 연동해주세요
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/My Drive/DSL/과제/CNN/scripts

/content/drive/My Drive/DSL/과제/CNN/scripts


In [4]:
# 이번 과제의 일부였던 CustomDataset.py, Model.py를 로드해주세요
from CustomDataset import CustomImageDataset
from Model import ResNet50

import pandas as pd
import numpy as np

import yaml
import os

import torch
import torch.optim as optim
import torch.nn as nn
import torch.functional as F

from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [5]:
# config.yaml의 위치를 지정해주세요
config_path = "/content/drive/My Drive/DSL/과제/CNN/scripts/config.yaml"

# 1. 하이퍼파라미터 및 기타 주요 변수 설정

config.yaml에 지정해줬던 디렉토리, 모델 하이퍼파라미터들을 가져옵시다

In [6]:
# config.yaml에서 디렉토리, 모델 하이퍼파라미터를 각각 받아옵니다
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

dataset_direc, meta_direc, weight_direc \
    = config["Direc"]["DatasetDirec"], \
      config["Direc"]["MetaDataDirec"], \
      config["Direc"]["WeightDirec"]

epoch, batch_size, lr \
    = config["HyperParameters"]["Epoch"], \
      config["HyperParameters"]["BatchSize"], \
      float(config["HyperParameters"]["LearningRate"]) # config.yaml에서 이미 float 자료형으로 학습률을 정의하신 분은 float()으로 자료형 변환을 하지 않으셔도 됩니다.



In [ ]:
pip install wandb --upgrade

In [8]:
import wandb # wandb.ai으로의 실시간 연동을 위한 라이브러리 임포트
from signal import signal, SIGPIPE, SIG_DFL
#Ignore SIG_PIPE and don't throw exceptions on it... (http://docs.python.org/library/signal.html)
signal(SIGPIPE,SIG_DFL)

# wandb login
wandb.init(
    project="DSL_ResNet", entity="yewonsong129",
    name="Brain Tumor (No_Augment)",
    config={
        "model_name": "Brain Tumor (No_Augment)", # augmentation을 적용한 것과 다른 이름을 주도록 하세요 :)
        "epochs": epoch,
        "batch_size": batch_size,
        "lr": lr
    })

config = wandb.config

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [9]:
# 가속을 위해 device로 GPU로 설정해줍니다
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# 2. train, val, test DataLoader 초기화

In [11]:
# MetaData.csv를 로드해주고, train, val, test로 나눠줍니다
meta_data = pd.read_csv('MetaData.csv')

# train : val : test = 6 : 2 : 2
train, val_test = train_test_split(meta_data, train_size=0.6, random_state=777)
val, test = train_test_split(val_test, train_size=0.5, random_state=777)

# CustomDataset.py에서 가져온 CustomImageDataset 클래스에 train, val, test 데이터프레임을 패스하여 데이터셋을 만들어주세요
train_dataset = CustomImageDataset(dataset_direc=dataset_direc, df=train, transform = None)
val_dataset   = CustomImageDataset(dataset_direc=dataset_direc, df=val, transform = None)
test_dataset  = CustomImageDataset(dataset_direc=dataset_direc, df=test, transform = None)


train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size = batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size = batch_size, shuffle=True)

# 3. 모델 초기화, 손실함수 및 optimizer 지정

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np

"""
[Batch, Channel, 224, 224]의 input을 받았을 때,
최종적으로 sigmoid activation function에 들어갈 값을 반환하는 ResBottleNeck, ResNet50 Class를 정의해주세요.
즉, 정의된 ResNet50의 output 값은 확률이 아니라 sigmoid에 들어가는 값이 됩니다.
"""

class ResBottleNeck(nn.Module):
    def __init__(self, in_c, middle_c, out_c, stage, first_bottle):
        super(ResBottleNeck, self).__init__()

        if stage == 2 and first_bottle == True:
            self.conv_layer1 = nn.Conv2d(in_c, in_c, kernel_size=1, stride=1, padding=0)
            self.bn_layer1 = nn.BatchNorm2d(in_c)

            self.conv_layer2 = nn.Conv2d(in_c, in_c, 3, 1, 1)
            self.bn_layer2 = nn.BatchNorm2d(in_c)

            self.conv_layer3 = nn.Conv2d(in_c, out_c, 1, 1, 0)
            self.bn_layer3 = nn.BatchNorm2d(out_c)

            self.skip_conn = nn.Conv2d(in_c, out_c, 1, 1, 0)
            self.bn_skip_conn = nn.BatchNorm2d(out_c)

        elif stage != 2 and first_bottle == True:
            self.conv_layer1 = nn.Conv2d(in_c, middle_c, kernel_size = 1, stride = 2, padding = 0)
            self.bn_layer1 =  nn.BatchNorm2d(in_c//2)

            self.conv_layer2 = nn.Conv2d(middle_c, middle_c, 3, 1, 1)
            self.bn_layer2 = nn.BatchNorm2d(in_c//2)

            self.conv_layer3 = nn.Conv2d(middle_c, out_c, 1, 1, 0)
            self.bn_layer3 = nn.BatchNorm2d(out_c)

            self.skip_conn = nn.Conv2d(in_c, out_c, 1, 2, 0)
            self.bn_skip_conn = nn.BatchNorm2d(out_c)

        if first_bottle == False :
            self.conv_layer1 = nn.Conv2d(in_c, middle_c, 1, 1, 0)
            self.bn_layer1 = nn.BatchNorm2d(middle_c)

            self.conv_layer2 = nn.Conv2d(middle_c, middle_c, 3, 1, 1)
            self.bn_layer2 = nn.BatchNorm2d(middle_c)

            self.conv_layer3 = nn.Conv2d(middle_c, out_c, 1, 1, 0)
            self.bn_layer3 = nn.BatchNorm2d(out_c)

            self.skip_conn = nn.Identity()
            self.bn_skip_conn = nn.BatchNorm2d(out_c)

        self.relu = nn.ReLU()
        self.first_bottle = True

    def forward(self, x):
        x_clone = self.bn_skip_conn(self.skip_conn(x))

        x = self.bn_layer1(self.conv_layer1(x))
        x = self.relu(x)

        x = self.bn_layer2(self.conv_layer2(x))
        x = self.relu(x)

        x = self.bn_layer3(self.conv_layer3(x))
        x = x + x_clone
        x = self.relu(x)

        return x


class ResNet50(nn.Module):
    def __init__(self, Bottleneck, num_classes, num_channels):
        super(ResNet50, self).__init__()

        #Stage 1
        self.conv_layer1 = nn.Conv2d(num_channels, 64, kernel_size = 7, stride = 2, padding = 3)
        self.bn_layer1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1, dilation = 1)

        #Stage 2 (3 layers)
        self.stage2 = nn.Sequential(
            Bottleneck(in_c = 64, middle_c = 64, out_c = 256, stage = 2, first_bottle = True),
            Bottleneck(256, 64, 256, stage = 2, first_bottle = False),
            Bottleneck(256, 64, 256, stage = 2, first_bottle = False)
          )

        #Stage 3 (4 layers)
        self.stage3 = nn.Sequential(
            Bottleneck(256, 128, 512, stage = 3, first_bottle = True),
            Bottleneck(512, 128, 512, stage = 3, first_bottle = False),
            Bottleneck(512, 128, 512, stage = 3, first_bottle = False),
            Bottleneck(512, 128, 512, stage = 3, first_bottle = False),
        )

        #Stage4 (6 layers)
        self.stage4 = nn.Sequential(
            Bottleneck(512, 256, 1024, stage = 4, first_bottle = True),
            Bottleneck(1024, 256, 1024, stage = 4, first_bottle = False),
            Bottleneck(1024, 256, 1024, stage = 4, first_bottle = False),
            Bottleneck(1024, 256, 1024, stage = 4, first_bottle = False),
            Bottleneck(1024, 256, 1024, stage = 4, first_bottle = False),
            Bottleneck(1024, 256, 1024, stage = 4, first_bottle = False)
        )

        #Stage5 (3 layers)
        self.stage5 = nn.Sequential(
            Bottleneck(1024, 512, 2048, stage = 5, first_bottle = True),
            Bottleneck(2048, 512, 2048, stage = 5, first_bottle = False),
            Bottleneck(2048, 512, 2048, stage = 5, first_bottle = False)
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(2048, num_classes)

    def forward(self,x):
        x = self.bn_layer1(self.conv_layer1(x))
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.stage5(x)

        x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [ ]:
# model을 선언하고 GPU에 올려줍니다
#from Model import ResNet50, ResBottleNeck
model = ResNet50(ResBottleNeck, num_channels = 1, num_classes = 1)
model.to(device)

In [16]:
# 손실함수를 정의해주세요
criterion = nn.BCEWithLogitsLoss()
# optimizer를 정의해주세요
optimizer = optim.Adam(model.parameters(), lr=lr)

# 4. Training 시작

학습을 위한 모든 준비가 끝났습니다! ㄱㅂㅈㄱ

In [17]:
# Best Validation Loss를 양의 무한대로 초기화
best_loss = float("inf")


# Training, Validation 시작

for curr_epoch in range(epoch+1): # iterate over all EPOCHS
  train_loss = 0
  correct_train = 0

  for batch_no, (img, label) in enumerate(train_loader): # iterate over all BATCHES

    # GPU에 올라가는 건 모델 뿐 아니라 데이터도 같이 올라가야합니다!
    img, label = img.to(device), label.to(device)

    # Forward Propagation
    pred = model(img)

    # 손실함수 계산
    label = label.view(-1, 1).float()
    loss = criterion(pred, label) # 구현상의 차이로 label 텐서에 조작을 가해 모양을 맞춰줘야하는 경우가 생길 수 있습니다 ( 힌트 : tensor.view() )

    # 누적된 기울기 초기화 및 역전파
    optimizer.zero_grad()
    loss.backward()

    # 파라미터 업데이트
    optimizer.step()

    # 해당 Batch에서의 loss를 train_loss 변수에 누적
    train_loss += loss.item()

    # Accuracy 계산
    pred_class = (torch.sigmoid(pred) > 0.5).float()
    correct_train += (pred_class == label).float().sum()

  # 모든 Batch를 순회한 이후 최종적인 train_loss, train_acc를 계산해줍니다
  train_loss = train_loss/len(train_loader.dataset)
  train_acc = correct_train /len(train_loader.dataset)

  # Validation 시작
  model.eval()

  with torch.no_grad():  # 학습 중이 아니므로 역전파를 위한 gradient 연산은 필요하지 않습니다
    val_loss = 0
    correct_val = 0

    for batch_no, (img, label) in enumerate(val_loader): # iterate over all BATCHES

      img,label = img.to(device), label.to(device)

      ##################################
      # forward propagation, loss 구현  #
      pred = model(img)
      label = label.view(-1, 1).float()
      ##################################

      loss = criterion(pred,label)
      val_loss += loss.item()

      # 모든 Batch를 순회한 이후 최종적인 val_loss, val_acc를 계산해줍니다
      pred_class = (torch.sigmoid(pred) > 0.5).float()
      correct_val += (pred_class == label).float().sum()

    val_loss /= len(val_loader.dataset)
    val_acc = correct_val / len(val_loader.dataset)

  # wandb.ai에 보낼 log 데이터를 작성해줍니다.
  metrics = {
    "epoch": epoch,
    "train_loss": train_loss,
    "val_loss" : val_loss,
    "train_acc": train_acc.item(),
    "val_acc" : val_acc.item(),
    "learning_rate": lr
    }

  # 아래 코드를 통해 실시간으로 학습현황을 확인할 수 있도록 log 데이터를 전송해줍니다
  wandb.log(metrics)

  # 모델을 다시 학습모드로 설정합니다
  model.train()

  # 가장 작은 validation loss를 가졌을 때의 모델 파라미터를 저장해줍니다.
  if val_loss < best_loss:
    best_loss = val_loss
    torch.save(model.state_dict(), os.path.join(weight_direc, "best_model(no aug).pt")) # .pt 확장자로 저장해주도록합니다

wandb.finish()